## A3. Community detection

### INFO

#### TASKS: 


[1] Apply at least three different community detection algorithms for the attached undirected networks

[2] At least one of the algorithms must be based on the optimization of modularity

[3] You must use at least two different programs

#### Comparissons: 
[1] Partition of reference, obtained from external information. In these cases, you have to compare your partitions with them, using at least the following standard measures: Jaccard Index, Normalized Mutual Information (arithmetic normalization), and Normalized Variation of Information.

#### DELIVERY
[1] a plot with color-coded communities

[2] Brief description of the algorithms and the programs used.

[3] Selected parameters for each algorithm and/or network, and the scripts used (if any).

[4] A table with the comparison measures between your partitions and the reference ones, grouped by network.

[5] A table with the modularity values of all the partitions (including the reference ones), grouped by network.

[6] The obtained partitions, in Pajek format (*.clu)



#### CAVEATS: 
[1] The position of the nodes must not change for all the partitions of the same network.

[2] If the network contains coordinates for the nodes (e.g. airports_UW.net), use them to establish the position of the nodes. Otherwise, use a layout algorithm to distribute the nodes in the plane trying to minimize the number of links crossings (e.g., Kamada-Kawai, ForceAtlas, etc.). Circular layouts must not be used.



## Community 

- Groups of densely connected components in various networks. 
- Most widely used algorithm Girvan-Newman algorithm

#### Techniques 
- **Agglomerative**: start only with the nodes of the original graph. Edges are added in a specific manner, if they have a weight, stronger ones are prioritized over weaker ones. 

- **Divisive**: Remove edges from the original graph iteratively. Stronger edges are removed before weaker ones. 

### Imports & Settings

In [ ]:
## get the community module 
!pip3 install -qq python-louvain

In [ ]:
## autoreload 
%load_ext autoreload
%autoreload 2

### Imports

In [ ]:
## import libraries 
import networkx as nx 
import matplotlib.pyplot as plt
from community import community_louvain

In [ ]:
## helper functions 
from src.helpers.community import NetworkXCommunityAlgs
from src.helpers.config import config_dict
from src.helpers.helpers import get_file_dict, read_clu,lol2idx,dict_vals_to_list,load_graph_coords
from src.helpers.metrics import (nmi,
                                 jaccard_index,
                                 rand_index,
                                 nvi_from_nmi)
from src.helpers.dataloader import get_net_clu_files, make_file_dict
from src.helpers.plotters import plot_graph_partition_original

### Defining Paths & Variables

In [ ]:
## Setting the PATHS to the specific directories 
DATA_DIR = './data'
IMG_DIR = './imgs'
## Loading the config dictionary 
CONFIG = config_dict(dir=DATA_DIR)
## getting all the net & clu files 
NET_FILES = list(map(get_net_clu_files, CONFIG.values()))
## make a dictionary to hold them 
FILE_DICT = list(map(make_file_dict, NET_FILES))
## add the NAMES items to the dictionary 
FILE_DICT = dict(zip(CONFIG.keys(), FILE_DICT))

### Plotting Original Graphs & Partitions

In [ ]:
## some settings 
FIGURE_SIZE = (20,10)
VISUALIZE = False # change this if you want to visualize the plots while they are being generated 
## grab one model-type and plot original graph & partition
#net_type = 'toy'
#tst = FILE_DICT[net_type]
## plotting ## some come out wrong even reading the coordinates
## these are plotted with igraph 
for net_type in FILE_DICT.keys():
    plot_graph_partition_original(data = FILE_DICT[net_type], 
                                net_type = net_type,
                                data_dir = DATA_DIR,
                                figure_size = FIGURE_SIZE,
                                save_dir = IMG_DIR,
                                visualize=VISUALIZE)

In [ ]:
## PLOTTING THE ONES THAT DON'T WORK WITH iGraph

## Grid 6x6 
dd = './data/toy/grid-p-6x6.net'
save_dir = "./imgs/toy/network_GRID_P_6x6_.png"
g = ig.read(dd)
visual_style = {}
visual_style["edge_width"] = 0.05
visual_style["vertex_size"] = 3
visual_style["bbox"] = (300,300)
visual_style["margin"] = 10
ig.plot(g, save_dir, **visual_style)

## AIRPORTS UW
## saving the AIRPORTS IMAGE 
dy = "./data/real/airports_UW.net"
save_dir = "./imgs/real/network_AIRPORTS_UW_.png"
g = ig.read(dy)
visual_style = {}
visual_style["edge_width"] = 0.05
visual_style["vertex_size"] = 3
visual_style["bbox"] = (720,480)
visual_style["margin"] = 10
ig.plot(g, save_dir, **visual_style)


### Calculating Partitions for each graph

In [ ]:
"""
Go over the keys in the dictionary 
filter the dictionary 
get the original file_name 
partition algorithm
graph the partitions
## for each partition for the graph g1.p1 calculate the metrics 
g1.p1 vs g1.c1, etcc where g1.c1 is the calculated partition 
"""

In [ ]:
for x,y in FILE_DICT[net_type].items():
    print(x,y)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = (12,8)
## get a single model with its partitions 
net_type = 'toy'
model_name = 'graph3+1+3'
## original directory 
og_dir = f"{DATA_DIR}/{net_type}/{model_name}.net"
## load the graph and the position 
g, pos = load_graph_coords(file_path = og_dir)
## read the clu file 
clu_file = FILE_DICT[net_type][model_name][0]
cl = dict(read_clu(clu_file))

In [ ]:
## iterate over the model types 
for net_type,networks in FILE_DICT.items():
    ## iterate over the networks 
    for graph, partition in networks.items():
        ## original directory 
        og_dir = f"{DATA_DIR}/{net_type}/{graph}.net"
        ## load the graph and the position 
        g, pos = load_graph_coords(file_path = og_dir)
        ## get the length of the partition 
        psize = len(partition)
        ## if it only has one partition
        if psize==1: 
            ## load the only partition 
            clu_file = partition[0]
            cl = dict(read_clu(clu_file))
            ## we plot the 1x5 graph (original + 4x methods)
            ## if there are multiple partitions, get the corresponding one and plot them 
            fig, axs = plt.subplots(1, 5, figsize=(20,8)) ## maybe change to 2x3 lets see.
            axs = axs.ravel()
            ## plot 1: Original Partition 
            nx.draw(g,pos=pos, ax=axs[0], node_color=list(cl.values()), node_size=15, alpha=0.9)
            axs[0].set_title(f"Number of Communities: {len(set([x for x in cl.values()]))}")

            ## PARTITION 1: NEWMAN
            ca1 = NetworkXCommunityAlgs(g, method='girvan_newman',layout=pos, verbosity=False)
            ca1_comm, ca1_nc = ca1.algorithm
            nx.draw(g,pos=pos, node_color=dict_vals_to_list(ca1_nc[0]), node_size=15, alpha=0.9,ax=axs[1])
            axs[1].set_title(f"{ca1.method.upper()} | Communities: {len(set([x for x in ca1_nc[0].values()]))}")

            ## plot the method 2: ASYN_FLUID: PARAMETERS: k=5, max_iter=100
            params = {'_k':5, '_max_iter':100}
            ca2 = NetworkXCommunityAlgs(g, method='asyn_fluid',layout=pos, verbosity=False, params=params)
            ca2_comm, ca2_nc = ca2.algorithm
            nx.draw(g,pos=pos, ax=axs[2], node_color=dict_vals_to_list(ca2_nc), node_size=15, alpha=0.9)
            axs[2].set_title(f"{ca2.method.upper()} | Communities: {len(set([x for x in ca2_nc.values()]))}")

            ## plotting the method 3: Label Propagation
            ca3 = NetworkXCommunityAlgs(g, method='label_prop',layout=pos, verbosity=False)
            ca3_comm, ca3_nc = ca2.algorithm
            nx.draw(g,pos=pos, ax=axs[3], node_color=dict_vals_to_list(ca3_nc), node_size=15, alpha=0.9)
            axs[3].set_title(f"{ca3.method.upper()} | Communities: {len(set([x for x in ca3_nc.values()]))}")

            ## plotting the method 4: CN-Moore  
            params = {'n_comm':3} ## number of communities 
            ca4 = NetworkXCommunityAlgs(g, method='cn_moore',layout=pos, verbosity=False, params=params)
            ca4_comm, ca4_nc = ca4.algorithm
            nx.draw(g,pos=pos, ax=axs[4], node_color=dict_vals_to_list(ca4_nc), node_size=15, alpha=0.9)
            axs[4].set_title(f"{ca4.method.upper()} | Communities: {len(set([x for x in ca4_nc.values()]))}")

        plt.show()
            
            

In [ ]:

## if there are multiple partitions, get the corresponding one and plot them 
fig, axs = plt.subplots(1, 5, figsize=(20,8))
axs = axs.ravel()
## plot 1: Original Partition 
nx.draw(g,pos=pos, ax=axs[0], node_color=list(cl.values()), node_size=15, alpha=0.9)
axs[0].set_title(f"Number of Communities: {len(set([x for x in cl.values()]))}")

## PARTITION 1: NEWMAN
ca1 = NetworkXCommunityAlgs(g, method='girvan_newman',layout=pos, verbosity=False)
ca1_comm, ca1_nc = ca1.algorithm
nx.draw(g,pos=pos, node_color=dict_vals_to_list(ca1_nc[0]), node_size=15, alpha=0.9,ax=axs[1])
axs[1].set_title(f"{ca1.method.upper()} | Communities: {len(set([x for x in ca1_nc[0].values()]))}")

## plot the method 2: ASYN_FLUID: PARAMETERS: k=5, max_iter=100
params = {'_k':5, '_max_iter':100}
ca2 = NetworkXCommunityAlgs(g, method='asyn_fluid',layout=pos, verbosity=False, params=params)
ca2_comm, ca2_nc = ca2.algorithm
nx.draw(g,pos=pos, ax=axs[2], node_color=dict_vals_to_list(ca2_nc), node_size=15, alpha=0.9)
axs[2].set_title(f"{ca2.method.upper()} | Communities: {len(set([x for x in ca2_nc.values()]))}")

## plotting the method 3: Label Propagation
ca3 = NetworkXCommunityAlgs(g, method='label_prop',layout=pos, verbosity=False)
ca3_comm, ca3_nc = ca2.algorithm
nx.draw(g,pos=pos, ax=axs[3], node_color=dict_vals_to_list(ca3_nc), node_size=15, alpha=0.9)
axs[3].set_title(f"{ca3.method.upper()} | Communities: {len(set([x for x in ca3_nc.values()]))}")

## plotting the method 4: CN-Moore  
params = {'n_comm':3} ## number of communities 
ca4 = NetworkXCommunityAlgs(g, method='cn_moore',layout=pos, verbosity=False, params=params)
ca4_comm, ca4_nc = ca4.algorithm
nx.draw(g,pos=pos, ax=axs[4], node_color=dict_vals_to_list(ca4_nc), node_size=15, alpha=0.9)
axs[4].set_title(f"{ca4.method.upper()} | Communities: {len(set([x for x in ca4_nc.values()]))}")

## showing
plt.tight_layout()
plt.show()

In [ ]:
ca1.method

In [ ]:
## iterate over the networks and get their corresponding partitions 
## except AIRPORTS & Grid 6x6
save_dir = "./imgs/partitions/"
## test a single model 
net_type = 'toy'
tst = FILE_DICT[net_type]
## original path 
#og_dir = f"{DATA_DIR}/{net_type}/{k}.net"
for model,partitions in tst.items():
    og_dir = f"{DATA_DIR}/{net_type}/{model}.net"
    ## load the original graph 
    g, pos = load_graph_coords(file_path = og_dir)
    ## define the subplots 
    fig, axs = plt.subplots(1,4)
    axs = axs.ravel()
    ## load the partitions
    for idx,parts in enumerate(partitions):
        ## read the partition file 
        cl = dict(read_clu(parts))
        ## plot the partition method 1: Girvan-Newman
        ca1 = NetworkXCommunityAlgs(g, method='girvan_newman',layout=pos, verbosity=False)
        ca1_comm, ca1_nc = ca1.algorithm
        nx.draw(g,pos=pos, ax=axs[idx], node_color=dict_vals_to_list(ca1_nc[0]), node_size=15, alpha=0.9)
    plt.show()
    
    

In [ ]:
## iterate through the DATA dictionary and plot the graph and its corresponding partition
for model_type, netpart in FILE_DICT.items():
    ## iterate through each of the netpart files; tuples (.net, .clu)
    ## create the figure MODELS x PARTITIONS
    fig, axs = plt.subplots(len(FILE_DICT[model_type])+1, 2+4, figsize=(20,10),sharex=True, sharey=True)
    #axs = axs.ravel()
    for idx,pairs in enumerate(netpart): 
        _net, _clu = pairs #separate them
        net_name = _net.split("/")[-1].split(".")[0]
        g, pos = load_graph_coords(_net) # load the graph and layout
        cl = dict(read_clu(_clu)) # original partition 
        ## plot the original 
        nx.draw(g,pos=pos, ax=axs[idx,0], node_size=15, alpha=0.9)
        ## plot the partition
        nx.draw(g,pos=pos, ax=axs[idx,1], node_color=list(cl.values()), node_size=15, alpha=0.9)
        ## plot the partition method 1: Girvan-Newman
        ca1 = NetworkXCommunityAlgs(g, method='girvan_newman',layout=pos, verbosity=False)
        ca1_comm, ca1_nc = ca1.algorithm
        nx.draw(g,pos=pos, ax=axs[idx,2], node_color=dict_vals_to_list(ca1_nc[0]), node_size=15, alpha=0.9)
        ## plot the method 2: ASYN_FLUID: PARAMETERS: k=5, max_iter=100
        params = {'_k':5, '_max_iter':100}
        ca2 = NetworkXCommunityAlgs(g, method='asyn_fluid',layout=pos, verbosity=False, params=params)
        ca2_comm, ca2_nc = ca2.algorithm
        nx.draw(g,pos=pos, ax=axs[idx,3], node_color=dict_vals_to_list(ca2_nc), node_size=15, alpha=0.9)
        ## plotting the method 3: Label Propagation
        ca3 = NetworkXCommunityAlgs(g, method='label_prop',layout=pos, verbosity=False)
        ca3_comm, ca3_nc = ca2.algorithm
        nx.draw(g,pos=pos, ax=axs[idx,4], node_color=dict_vals_to_list(ca3_nc), node_size=15, alpha=0.9)
        ## plotting the method 4: CN-Moore  
        params = {'n_comm':3} ## number of communities 
        ca4 = NetworkXCommunityAlgs(g, method='cn_moore',layout=pos, verbosity=False, params=params)
        ca4_comm, ca4_nc = ca4.algorithm
        nx.draw(g,pos=pos, ax=axs[idx,5], node_color=dict_vals_to_list(ca4_nc), node_size=15, alpha=0.9)
        ## set the titles
        axs[idx,0].set_title(f'NET: {net_name}')
        axs[idx,1].set_title('Original Partition')
        axs[idx,2].set_title('Partition 1: Girvan-Newman')
        axs[idx,3].set_title('Partition 2: Asynchronous-Fluid')
        axs[idx,4].set_title('Partition 3: Label Propagation')
        axs[idx,5].set_title('Partition 4: CN-Moore')
    plt.suptitle(f'Model: {model_type}', fontsize=20, y=1.05)
    plt.tight_layout()
    #plt.savefig("figures/{}.png".format(model_type), bbox_inches='tight')
    plt.show()

### Testing iGraph Algorithms

In [ ]:
import igraph as ig
from pylab import rcParams
rcParams['figure.figsize'] = 100, 100
g = ig.read(dy)
## try to use the same methods we used 
## MODULARITY 
dendrogram = g.community_fastgreedy()
clusters = dendrogram.as_clustering()
membership = clusters.membership
## dictionary 
d = dict(zip(g.vs['name'],membership))
## plotting the colors
pal = ig.drawing.colors.ClusterColoringPalette(len(clusters))
g.vs['color'] = pal.get_many(clusters.membership)

visual_style = {}
visual_style["edge_width"] = 0.05
visual_style["vertex_size"] = 3
visual_style["bbox"] = (1920,1024)
visual_style["margin"] = 10
#ig.plot(g, **visual_style)

In [ ]:
## load the net and clu files
#g = nx.read_pajek(net)
g, pos = load_graph_coords(net)
cl = dict(read_clu(clu))

## plotting 
## create the figure 
fig, axs = plt.subplots(1, 2+NUM_PARTITIONS, figsize=(20,5))
axs = axs.ravel()
## plot the original 
nx.draw(g,pos=pos, ax=axs[0], node_size=15, alpha=0.9)

## plot the partition
nx.draw(g,pos=pos, ax=axs[1], node_color=list(cl.values()), node_size=15, alpha=0.9)


## plot the partition method 1: Girvan-Newman
ca1 = NetworkXCommunityAlgs(g, method='girvan_newman',layout=nx.kamada_kawai_layout(g), verbosity=False)
ca1_comm, ca1_nc = ca1.algorithm
nx.draw(g,pos=pos, ax=axs[2], node_color=dict_vals_to_list(ca1_nc[0]), node_size=15, alpha=0.9)


## plot the method 2: ASYN_FLUID: PARAMETERS: k=5, max_iter=100
params = {'_k':5, '_max_iter':100}
ca2 = NetworkXCommunityAlgs(g, method='asyn_fluid',layout=nx.kamada_kawai_layout(g), verbosity=False, params=params)
ca2_comm, ca2_nc = ca2.algorithm
nx.draw(g,pos=pos, ax=axs[3], node_color=dict_vals_to_list(ca2_nc), node_size=15, alpha=0.9)

## plotting the method 3: Label Propagation
ca3 = NetworkXCommunityAlgs(g, method='label_prop',layout=nx.kamada_kawai_layout(g), verbosity=False)
ca3_comm, ca3_nc = ca2.algorithm
nx.draw(g,pos=pos, ax=axs[4], node_color=dict_vals_to_list(ca3_nc), node_size=15, alpha=0.9)

## plotting the method 4: CN-Moore
params = {'n_comm':3} ## number of communities 
ca4 = NetworkXCommunityAlgs(g, method='cn_moore',layout=nx.kamada_kawai_layout(g), verbosity=False, params=params)
ca4_comm, ca4_nc = ca4.algorithm
nx.draw(g,pos=pos, ax=axs[5], node_color=dict_vals_to_list(ca4_nc), node_size=15, alpha=0.9)

## set the titles 
axs[0].set_title('Original')
axs[1].set_title('Partition')
axs[2].set_title('Partition 1: Girvan-Newman')
axs[3].set_title('Partition 2: Asynchronous-Fluid')
axs[4].set_title('Partition 3: Label Propagation')
axs[5].set_title('Partition 4: CN-Moore')

plt.suptitle(f'Toy Example: {net.split("/")[-1]}', fontsize=20, y=1.1)
plt.show()

### iGraph Algorithms

In [ ]:
import igraph as ig 
## load the sample 
g = ig.read(net)
## try to use the same methods we used 
## MODULARITY 
dendrogram = g.community_fastgreedy()
clusters = dendrogram.as_clustering()
membership = clusters.membership
## dictionary 
d = dict(zip(g.vs['name'],membership))
## plotting the colors
pal = ig.drawing.colors.ClusterColoringPalette(len(clusters))
g.vs['color'] = pal.get_many(clusters.membership)
ig.plot(g)

In [ ]:
## LABEL PROPAGATION 
lp = g.community_label_propagation()
lp_mem = lp.membership
## dictionary
dlp = dict(zip(g.vs['name'],lp_mem))
lp_pal = ig.drawing.colors.ClusterColoringPalette(len(lp))
g.vs['color'] = lp_pal.get_many(lp.membership)
ig.plot(g)

In [ ]:
## GIRVAB-NEWMAN
gn = g.community_leading_eigenvector()
gn_mem = lp.membership
## dictionary
dlp = dict(zip(g.vs['name'],gn_mem))
gn_pal = ig.drawing.colors.ClusterColoringPalette(len(gn))
g.vs['color'] = gn_pal.get_many(gn.membership)
ig.plot(g)

In [ ]:
## This works to plot the different k-communities in a graph 
from networkx.algorithms.community.centrality import girvan_newman
mapper = {name:idx for idx, name in  enumerate(ga.nodes())}
## apply the algorithm to the graph 
communities = girvan_newman(ga) ## generator object
## another approach to get the community
pos = nx.fruchterman_reingold_layout(ga)
import itertools 
## define the number of communities (tuple)
k = 6
fig, axs = plt.subplots(1, k, figsize=(20,5))
axs = axs.ravel()
for idx, comm in enumerate(itertools.islice(communities, k)):
    part = tuple(sorted(c) for c in comm)
    b = lol2idx(part) ## Converts the list of list into a dictionary of sublist-index 
    nx.draw(ga,pos=pos, node_color=list(b.values()), node_size=15, ax=axs[idx])#,with_labels=True)
    axs[idx].set_title(f"Partition k={idx+1}")
plt.tight_layout()

### NetworkX Algorithms

#### Community Detection Algorithm 1: Girvan-Newman algorithm

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.centrality.girvan_newman.html#networkx.algorithms.community.centrality.girvan_newman

- Divisive method, progressively removes edges from the original graph. 
- Removes the "most-valuable" edge. 
    - Highest betweenness centrality.
        - Highest number of shortest paths between nodes. 
        - definition is unclear but something as: 
            Number of shortest paths through V or E / Total shortest paths
- Result can be shown as a dendrogram. 

In [ ]:
## apply the algorithm to the graph 
communities = girvan_newman(g) ## generator object 
## get the nodes belonging to the first community
node_groups = [com for com in next(communities)]
## loop over the nodes in the original graph 
## if they are the original assign a color to them
c1 = 'red'
c2 = 'blue'
color_map = [c2 if node in node_groups[0] else c1  for node in g.nodes()]
## draw it again 
nx.draw(g, node_color=color_map, with_labels=True)
plt.show()

In [ ]:
import networkx.algorithms.community as nx_comm
nx_comm.modularity(g, node_groups)

In [ ]:
## another approach to get the community
import itertools 
## define the number of communities (tuple)
k = 5
for comm in itertools.islice(communities, k):
    print(tuple(sorted(c) for c in comm))

In [ ]:
## using another package 
from community import community_louvain
import community
import matplotlib.cm as cm

# compute the best partition
partition = community_louvain.best_partition(g)
print(partition)
# draw the graph
pos = nx.spring_layout(g)
# color the nodes according to their partition
cmap = cm.get_cmap('tab10', max(partition.values()) + 1)
## draw the nodes
nx.draw_networkx_nodes(g, pos, partition.keys(), node_size=40,
                       cmap=cmap, node_color=list(partition.values()))
## draw the edges 
nx.draw_networkx_edges(g, pos, alpha=0.5)
plt.show()


In [ ]:
## given two sets of partitions or communities, calculate the metrics

## community 1 
l1 = sorted(partition.values(),reverse=False)
## community 2 
l2 = [1,1,1,1,2,2,2,2]
## normalized mutual information
_nmi = nmi(l1,l2)
## jaccard index 
jac_idx = jaccard_index(l1,l2)
## randindex 
rand_id = rand_index(l1,l2)
## nvi = normalized variation of information
_nvi = nvi_from_nmi(l1,l2,len(l1))

## feedback 
print(f"NMI: {_nmi:.2f}")
print(f"Jaccard index: {jac_idx:.2f}")
print(f"Rand index: {rand_id:.2f}")
print(f"NVI: {_nvi:.2f}")
tracker_metrics = {'nmi':_nmi,
                   'jaccard_index':jac_idx,
                   'rand_index':rand_id,
                   'nvi':_nvi}

In [ ]:
partition = community.best_partition(g)
pos = nx.spring_layout(g)
nx.draw_networkx_nodes(g, pos, node_color=list(partition.values))
nx.draw_networkx_edges(g,pos,alpha=0.3)

#### Community Detection Algorithm 2: Fluid Communities algorithm

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.centrality.girvan_newman.html#networkx.algorithms.community.centrality.girvan_newman


Very nice graphs 
https://arxiv.org/pdf/1703.09307.pdf

- Based on "fluids interacting with each other" 
    - expanding and pushing each other. 

Mechanics: 
- Initial k communities initialized on a random vertex. 
- Iterate over all vertices in random order. 
- **Vertex move from one place to the other, different densities, shifts**


In [ ]:
## import 
from networkx.algorithms.community import asyn_fluid
## set parameters 
## K is the number of communities
K = 6
## max_iter is the maximum number of iterations
max_iter = 100
## assign fluids 
fluids = asyn_fluid.asyn_fluidc(G=g, k=K, max_iter=max_iter)
## iterate over the fluids and print the communities
cmms = [fluid for fluid in fluids]



fig, axs = plt.subplots(3,2, figsize=(20,10))
axs = axs.ravel()
## assign a color to each community
for idx, i in enumerate(cmms):
    color_map = [c2 if node in cmms[idx] else c1  for node in g.nodes()]     
    ## drawing the graph
    nx.draw(g, node_color=color_map, with_labels=True, ax=axs[idx])

In [ ]:
fig, axs = plt.subplots(4,2, figsize=(20,10))
axs = axs.ravel()
for idx, K in enumerate(range(1,len(g.edges()),1)):
    fluids = asyn_fluid.asyn_fluidc(G=g, k=K, max_iter=max_iter)
    l = [fluid for fluid in fluids]
    cy = lol2idx(l)
    cd = dict_vals_to_list(cy)
    nx.draw(g, node_color=cd, with_labels=True, ax=axs[idx])
plt.tight_layout()

#### Community Detection Algorithm 3: Label Propagation algorithm

In [ ]:
from networkx.algorithms.community import label_propagation
#compute the communities
communities = label_propagation.label_propagation_communities(g)
nc = lol2idx(communities)
nx.draw(g, node_color=dict_vals_to_list(nc), with_labels=True)

#### Community Detection Algorithm 4: Clique Percolation algorithm

In [ ]:
from networkx.algorithms.community import k_clique_communities
## define k
k = 2
comms = list(k_clique_communities(g,k))
nc = lol2idx(comms)
nx.draw(g, node_color=dict_vals_to_list(nc), with_labels=True)
